In [4]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML 
import nltk

In [5]:
from collections import Counter

In [6]:
from nltk.tokenize import sent_tokenize

dvach = open('2ch_corpus.txt', encoding = 'UTF-8').read()
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

sentences_dvach = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)][:-100]
sentences_perplexity = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)][-100:]

In [8]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()

i = 0

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence))
    trigrams_dvach.update(ngrammer(sentence), n=3)
    if i >4000:
        break
    i += 1

In [9]:
matrix_dvach = np.zeros((len(bigrams_dvach), 
                   len(unigrams_dvach)))

id2word_dvach_1 = list(unigrams_dvach)
word2id_dvach_1 = {word:i for i, word in enumerate(id2word_dvach_1)}

id2word_dvach_2 = list(bigrams_dvach)
word2id_dvach_2 = {word:i for i, word in enumerate(id2word_dvach_2)}

In [21]:
from scipy.sparse import csr_matrix, csc_matrix, lil_matrix

matrix_dvach = lil_matrix((len(bigrams_dvach), len(unigrams_dvach)))

id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}

id2bigram_dvach = list(bigrams_dvach)
bigram2id_dvach = {word:i for i, word in enumerate(id2bigram_dvach)}

for ngram in trigrams_dvach:
    word1, word2, word3 = ngram.split()
    bigram = word1 + ' '+ word2 

    matrix_dvach[bigram2id_dvach[bigram], word2id_dvach[word3]] = (trigrams_dvach[ngram]/bigrams_dvach[bigram])

matrix_dvach = csr_matrix(matrix_dvach)

ValueError: not enough values to unpack (expected 3, got 2)

In [13]:
for ngram in trigrams_dvach:
    word1 = " ".join(ngram.split()[:2])
    word2 = ngram.split()[-1]
    matrix_dvach[word2id_dvach_2[word1]][word2id_dvach_1[word2]] = (trigrams_dvach[ngram]/
                                                                     bigrams_dvach[word1])
    
    def generate(matrix, id2word_2, id2word_1, word2id_2, n = 200, start='<start> <start>'):
        text = []
        current_idx = word2id_2[start]
        
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word_uni[chosen])
        
        if id2word_uni[chosen] == '<end>':
            current_idx = word2id_bi[start]
            
        else:
            part = id2word_2[current_idx] + ' ' + id2word_1[chosen]
            part = ' '.join(part.split()[1:])
            current_idx = word2id_bi[part]
    
return ' '.join(text)

NameError: name 'n' is not defined

In [14]:
from scipy.sparse import csr_matrix, csc_matrix, lil_matrix

In [15]:
matrix_dvach = lil_matrix((len(unigrams_dvach), 
                   len(unigrams_dvach)))

id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}

id2bigram_dvach = list(bigrams_dvach)
bigrams2id = {word:i for i, word in enumerate(id2bigram_dvach)}

for ngram in trigrams_dvach:
    word1 = " ".join(ngram.split()[:2])
    word2 = ngram.split()[-1]
    matrix_dvach[word2id_dvach[word1]][word2id_dvach[word2]] = (trigrams_dvach[ngram]/
                                                                     bigrams_dvach[word1])

KeyError: '<start> <start>'

In [16]:
def generate(matrix, id2word, word2id, n=200, start ='<start> <start>'):
    text = []
    current_idx = word2id['<start>']
    
    for i in range(n):
        
        chosen = np.random.choice(list(range(matrix.shape[1])), p=matrix[current_idx].toarray()[0])
        text.append(id2word[chosen])
        
        
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        current_idx = chosen
    
    return ' '.join(text)

In [17]:
print(generate(matrix_dvach, id2word_dvach, word2id_dvach, n=200).replace('<end>', '\n\n'))

ValueError: probabilities do not sum to 1

In [18]:
def perplexity(probas):
    p = np.exp(np.sum(probas))
    N = len(probas)
    
    return p**(-1/N) 

**каждый раз выдаёт новую ошибку**

**пробовала разные способы, в итоге так и не получилось(**